# Example code for students on the EDSA course

Ternary diagrams are great for plotting chemical compositions as they often contain more than two components. You will likely come across these yourself in geochemistry style courses so that you can compare melt or rock compositions, or even melt evolution.

If you haven't seen one before, ternary diagrams are 2D triangular plots where each apex marks 100% of the component at that apex. The axes then mark how much of that component is in the system, much like a normal rectangular graph. As ternary diagrams are used for three component systems, each point in the diagram will have three components. 

The example below shows how to create a ternary diagram using a package called Plotly. This was used to plot the Fe,Ni-FeS-Fe3P system in order to explore the compositions of immiscible liquids produced during melting conditions comparable to cores of planetesimals (small planetoid bodies).


## Create a ternary diagram

The first step of most coding is to import the relevant packages. Packages like numpy are useful to always import. Other packages might be required for the specific problem, for example, here, the plotly packages are required.

In [1]:
## IMPORT PACKAGES
#Import packages for number crunching and plotting graphs 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import minmax_scale

#Import packages to read in and organise files 
import pandas as pd 
import glob

#Import packages to plot chemical composition ternary diagrams 
import plotly.graph_objects as go
import plotly.figure_factory as ff


import warnings
warnings.filterwarnings('ignore')

The next step is to read in your datafiles. These can be done one by one, but that gets tedious if you have lots of data. Instead you can use the data below to grab the files you need. The data used here is in csv files, but other formats could be used by swapping out the ".csv" . 

In [2]:
## SELECTING DATAFILES

#If your files are in the same folder as the jupyter notebook, you do not need to specify a path
#If your files are in a different folder you will need to tell the code where the data is kept
#path=""
all_files=glob.glob('*.csv')
#Don't need the file called ALL_P.csv just yet. You will possibly see it in a later notebook.
all_files.remove("ALL_P.csv")

# Make an empty dictionary to hold all files 
data={}

#Loop through every file and read it in with pandas then add it to empty dictionary
for filename in all_files:
    data[filename]=pd.read_csv(filename)
    
all_files

['Steenstra20b.csv',
 'Steenstra20a.csv',
 'Rushmer05.csv',
 'McCoy99.csv',
 'Ford08.csv',
 'ChabotDrake00.csv']

Now we can start thinking about plotting the ternary diagram. The code below uses loops to make it more readable (and easier to debug!). As this was plotted in relation to composition of immiscible liquids, each file is split into two dataframes; one containing the metallic (sulphur-poor) phase, and the other containing the sulfide (sulphur-rich) phase. 

In [7]:
## PLOT TERNARY DIAGRAM

# Define proportions of elements in the end members FeS and Fe3P
Sprop = 36.47
Pprop = 15.60

# Define marker type and size
#The documentation for go.Scatterternary() has a list of colours and shapes that can be chosen
#They are different to those used in matplotlib
# 0 gives circles, 2 gives diamonds 
markers = [0, 2] 
size = 10


# Create an empty figure 
fig=go.Figure(go.Scatterternary())


# Loop through the files with the data from the different authors
for i in range(0, len(all_files)):
    
    # Select data for sulfides and metals
    sulfides = data[all_files[i]].loc[data[all_files[i]]["Phase"]=="sulfides"]
    metals = data[all_files[i]].loc[data[all_files[i]]["Phase"]=="metals"]
    
    # Create additional columns
    # Fe+ Ni - iron and nickel often group together in metallic phases
    sulfides["Fe+Ni"] = (sulfides["Fe"] + sulfides["Ni"]) 
    metals["Fe+Ni"] = (metals["Fe"] + metals["Ni"])
    
    # FeS - this changes the end member from pure S to FeS so that the data is not squeezed into a small area of the plot
    sulfides["FeS"] = sulfides["S"]*(100/Sprop) 
    metals["FeS"] = metals["S"]*(100/Sprop) 
    
    # Fe3P - this changes the end member from pure P to Fe3P
    sulfides["Fe3P"]=sulfides["P"]*(100/Pprop) 
    metals["Fe3P"]=metals["P"]*(100/Pprop) 
    
    
    # Scale the columns by the sum of the 3 elements (Fe+Ni, FeS, Fe3P) - this makes sure that the components add up to 100%
    sum_all = np.sum([sulfides["Fe+Ni"], sulfides["FeS"], sulfides["Fe3P"]])
    scaling = 100 / sum_all
    
    sulfides["Fe+Ni"] =  sulfides["Fe+Ni"]*scaling
    sulfides["FeS"] = sulfides["FeS"]*scaling
    sulfides["Fe3P"] =  sulfides["Fe3P"]*scaling
    
    metals["Fe+Ni"] = metals["Fe+Ni"]*scaling
    metals["FeS"] = metals["FeS"]*scaling
    metals["Fe3P"] = metals["Fe3P"]*scaling
    
    
    # Plot the ternary diagram by adding the sulfides and metals to the empty diagram set up before the loop
    fig.add_scatterternary(a=sulfides["FeS"], b=sulfides["Fe+Ni"], c=sulfides["Fe3P"], showlegend=False, 
                       mode="markers",marker=dict(symbol=markers[0], size=size))
    fig.add_scatterternary(a=metals["FeS"], b=metals["Fe+Ni"], c=metals["Fe3P"], showlegend=False, 
                           mode="markers", marker=dict(symbol=markers[1], size=size))
    
#####################################################################
#We are now outside the loop 

# Adjust the layout of the plot and show the figure 
def makeAxis(title, tickangle):
    return {'title': title, 'titlefont': { 'size': 20 }, 'tickangle': tickangle,
      'tickfont': { 'size': 15 }, 'tickcolor': 'rgba(0,0,0,0)', 'ticklen': 5,
      'showline': True, 'showgrid': True}

fig.update_layout({
    'ternary': {'aaxis': makeAxis('FeS', 0), 'baxis': makeAxis('<br>Fe+Ni', 0),'caxis': makeAxis('<br>Fe3P', 0)} })

fig.show()


## Create a density diagram

In [4]:
# LOAD DATA AND SCALE COLUMNS

# Define proportions of elements
Sprop = 36.47
Pprop = 15.60

# Read in file
all_p=pd.read_csv("ALL_P.csv")

# As before for Fe+Ni, FeS, Fe3P
all_p["Fe+Ni"] = all_p["Fe"] + all_p["Ni"]
all_p["FeS"] = all_p["S"] * (100/Sprop)
all_p["Fe3P"] = all_p["P"] * (100/Pprop)

# Scale by the sum 
all_p["sum"]=np.sum([all_p["Fe+Ni"], all_p["FeS"], all_p["Fe3P"]], axis=0)
all_p["Fe+Ni_new"] = all_p["Fe+Ni"] * (100/all_p["sum"])
all_p["FeS_new"]   = all_p["FeS"] * (100/all_p["sum"])
all_p["Fe3P_new"]  =  all_p["Fe3P"] * (100/all_p["sum"])

In [5]:
# PLOT DENSITY DIAGRAM
fig = ff.create_ternary_contour(np.array([all_p["FeS_new"], all_p["Fe+Ni_new"], all_p["Fe3P_new"]]), all_p["log fO2"],
                                pole_labels=['FeS', 'Fe+Ni', 'Fe3P'],
                                ncontours=10,
                                coloring="lines",
                                colorscale="Greys",
                                interp_mode='ilr',
                                showmarkers=True, showscale=True)

fig.update_traces(marker_colorbar_title='Log(fO2)', selector=dict(marker_showscale=True))
fig.show()